In [ ]:
import tensorflow as tf
import os

# Kiểm tra phiên bản TensorFlow
print(f"TensorFlow version: {tf.__version__}")

# Cấu hình memory growth để sử dụng GPU hiệu quả
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Tìm thấy {len(gpus)} GPU:")
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Cấu hình memory growth
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Một số tùy chọn để tối ưu hiệu suất cho GPU T4
        os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
        os.environ['TF_GPU_THREAD_COUNT'] = '2'  # Tương ứng với số GPU
        
        # Hiển thị các GPU logic
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"Số lượng GPU vật lý: {len(gpus)}, số lượng GPU logic: {len(logical_gpus)}")
        
        # Thông tin chi tiết về GPU
        from tensorflow.python.client import device_lib
        local_device_protos = device_lib.list_local_devices()
        gpu_list = [x.name for x in local_device_protos if x.device_type == 'GPU']
        print(f"Danh sách GPU: {gpu_list}")
        
        # Hiển thị thông tin CUDA và cuDNN
        build_info = tf.sysconfig.get_build_info()
        print(f"CUDA version: {build_info.get('cuda_version', 'N/A')}")
        print(f"cuDNN version: {build_info.get('cudnn_version', 'N/A')}")
        
        # Kiểm tra xem GPU có thực sự được sử dụng hay không
        print("\nXác nhận GPU đang hoạt động bằng phép tính nhỏ:")
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
            b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
            c = tf.matmul(a, b)
            print(f"Tính toán trên GPU: {c}")
            print(f"Đang chạy trên thiết bị: {c.device}")
    
    except RuntimeError as e:
        print(f"Lỗi khi cấu hình GPU: {e}")
else:
    print("Không tìm thấy GPU! Đang sử dụng CPU.")
    
    # Kiểm tra thông tin CPU
    cpu_devices = tf.config.list_physical_devices('CPU')
    print(f"Tìm thấy {len(cpu_devices)} CPU: {cpu_devices}")

In [ ]:
import glob
import matplotlib.pyplot as plt
import cv2
import random
import pandas as pd
import os
from PIL import Image
import warnings
import gc
import numpy as np
import csv
import time
import seaborn as sns
from tqdm import tqdm 
import shutil

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Conv2D, Flatten, Dropout, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, Resizing
from tensorflow.keras.layers import MaxPooling2D, Activation, BatchNormalization, Attention, Reshape, RepeatVector, Lambda, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import image

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import joblib

import onnxruntime as ort
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Bỏ qua các cảnh báo
warnings.filterwarnings("ignore")

# In phiên bản TensorFlow hiện tại
print('TensorFlow Version ' + tf.__version__)

def seed_everything(seed=0):
    # Thiết lập seed để đảm bảo tính tái lập (reproducibility)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Gọi hàm seed_everything để thiết lập seed mặc định
seed_everything()

In [1]:
image_size = 224
n_classes = 45
batch_size = 32

classes_train = {
    1: "21_Ngay_Yeu_Em",
    2: "4_Nam_2_Chang_1_Tinh_Yeu",
    3: "An_Tet_Ben_Con",
    4: "Bay_Ngot_Ngao",
    5: "Benh_Vien_Ma",
    6: "Bi_Mat_Lai_Bi_Mat",
    7: "Bi_Mat_Trong_Suong_Mu",
    8: "Bo_Tu_Oan_Gia",
    9: "Cho_Em_Den_Ngay_Mai",
    10: "Chu_Tich_Giao_Hang",
    11: "Chuyen_Tet",
    12: "Co_Ba_Sai_Gon",
    13: "Dao_Pho_Va_Piano",
    14: "Dat_Rung_Phuong_Nam",
    15: "Dia_Dao",
    16: "Dinh_Menh_Thien_Y",
    17: "Doi_Mat_Am_Duong",
    18: "Em_Chua_18",
    19: "Em_La_Cua_Em",
    20: "Gai_Gia_Lam_Chieu_3",
    21: "Gia_Ngheo_Gap_Phat",
    22: "Hem_Cut",
    23: "Hoan_Doi",
    24: "Ke_An_Danh",
    25: "Ke_An_Hon",
    26: "Lam_Giau_Voi_Ma",
    27: "Lat_Mat_1",
    28: "Lo_Mat",
    29: "Ma_Da",
    30: "Mat_Biec",
    31: "Nghe_Sieu_De",
    32: "Nhung_Nu_Hon_Ruc_Ro",
    33: "Ong_Ngoai_Tuoi_30",
    34: "Phap_Su_Tap_Su",
    35: "Quy_Cau",
    36: "Quy_Co_Thua_Ke",
    37: "Ra_Mat_Gia_Tien",
    38: "Sieu_Lua_Gap_Sieu_Lay",
    39: "Sieu_Tro_Ly",
    40: "Tam_Cam_Chuyen_Chua_Ke",
    41: "Taxi_Em_Ten_Gi",
    42: "The_Call",
    43: "Thien_Menh_Anh_Hung",
    44: "Tieu_Thu_Va_Ba_Dau_Gau",
    45: "Tren_Ban_Nhau_Duoi_Ban_Muu"
}

In [2]:
classes_test = {
    1: "21_Ngay_Yeu_Em",
    2: "4_Nam_2_Chang_1_Tinh_Yeu",
    3: "An_Tet_Ben_Con",
    4: "Bay_Ngot_Ngao",
    5: "Benh_Vien_Ma",
    6: "Bi_Mat_Lai_Bi_Mat",
    7: "Bi_Mat_Trong_Suong_Mu",
    8: "Bo_Tu_Oan_Gia",
    9: "Cho_Em_Den_Ngay_Mai",
    10: "Chu_Tich_Giao_Hang",
    11: "Chuyen_Tet",
    12: "Co_Ba_Sai_Gon",
    13: "Dao_Pho_Va_Piano",
    14: "Dat_Rung_Phuong_Nam",
    15: "Dia_Dao",
    16: "Dinh_Menh_Thien_Y",
    17: "Doi_Mat_Am_Duong",
    18: "Em_Chua_18",
    19: "Em_La_Cua_Em",
    20: "Gai_Gia_Lam_Chieu_3",
    21: "Gia_Ngheo_Gap_Phat",
    22: "Hem_Cut",
    23: "Hoan_Doi",
    24: "Ke_An_Danh",
    25: "Ke_An_Hon",
    26: "Lam_Giau_Voi_Ma",
    27: "Lat_Mat_1",
    28: "Lo_Mat",
    29: "Ma_Da",
    30: "Mat_Biec",
    31: "Nghe_Sieu_De",
    32: "Nhung_Nu_Hon_Ruc_Ro",
    33: "Ong_Ngoai_Tuoi_30",
    34: "Phap_Su_Tap_Su",
    35: "Quy_Cau",
    36: "Quy_Co_Thua_Ke",
    37: "Ra_Mat_Gia_Tien",
    38: "Sieu_Lua_Gap_Sieu_Lay",
    39: "Sieu_Tro_Ly",
    40: "Tam_Cam_Chuyen_Chua_Ke",
    41: "Taxi_Em_Ten_Gi",
    42: "The_Call",
    43: "Thien_Menh_Anh_Hung",
    44: "Tieu_Thu_Va_Ba_Dau_Gau",
    45: "Tren_Ban_Nhau_Duoi_Ban_Muu",
    46: "Khac"
}

In [7]:
import os
import cv2
import numpy as np
import pandas as pd
import time
import faiss
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from collections import Counter
import random

# ================== CONFIG ==================
image_size = 224
vector_length = 128  # SIFT descriptor length
random.seed(42)

train_path = 'E:\\Data\\Movie_Dataset\\Process_Frames_2\\Train'
test_path = 'E:\\Data\\Movie_Dataset\\Process_Frames_2\\Test'

index_path = "faiss_features.index"
label_path = "faiss_labels.npy"

confusion_output_path = "confusion_matrix_sift_faiss.jpg"
csv_output_path = "classification_report_sift_faiss.csv"
similarity_threshold = 0.8

# Mapping label (bạn cần định nghĩa sẵn dict này)
classes_train = classes_train
classes_test = classes_test

# ================== HÀM TIỆN ÍCH ==================
def l2_normalize(vectors):
    if vectors.ndim == 1:
        vectors = vectors.reshape(1, -1)
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / (norms + 1e-10)


def compute_rootsift(descriptors):
    eps = 1e-7
    descriptors = descriptors.astype(np.float32)
    l1_norm = np.linalg.norm(descriptors, ord=1, axis=1, keepdims=True)
    descriptors /= (l1_norm + eps)
    descriptors = np.sqrt(descriptors)
    return descriptors

def extract_sift_mean_vector(img_path, sift):
    img = cv2.imread(img_path)
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is None or len(descriptors) == 0:
        return None
    descriptors = compute_rootsift(descriptors)
    descriptors = l2_normalize(descriptors)
    mean_vector = np.mean(descriptors, axis=0).astype(np.float32)
    return mean_vector

# ================== PHẦN 1: TRÍCH XUẤT TRAIN & LƯU FAISS ==================
def build_faiss_index(train_path, classes):
    sift = cv2.SIFT_create()
    features_list, labels_list = [], []

    print("🧩 Trích xuất SIFT từ tập Train...")
    for label_id, label_name in classes.items():
        label_dir = os.path.join(train_path, label_name)
        if not os.path.isdir(label_dir):
            continue
        for filename in tqdm(os.listdir(label_dir), desc=f"Processing {label_name}"):
            img_path = os.path.join(label_dir, filename)
            mean_vector = extract_sift_mean_vector(img_path, sift)
            if mean_vector is None:
                continue
            features_list.append(mean_vector)
            labels_list.append(label_id - 1)

    features_array = np.array(features_list, dtype=np.float32)
    labels_array = np.array(labels_list)

    # Chuẩn hóa L2
    normalized_features = l2_normalize(features_array.astype('float32'))

    # FAISS index
    d = normalized_features.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(normalized_features)

    # Lưu
    faiss.write_index(index, index_path)
    np.save(label_path, labels_array)

    print(f"✅ Đã thêm {index.ntotal} vector vào FAISS.")
    return index, labels_array

def evaluate_faiss(test_path, classes, index, index_labels):
    sift = cv2.SIFT_create()
    class_to_idx = {name: idx for idx, name in classes.items()}
    idx_to_class = {idx: name for name, idx in class_to_idx.items()}

    # Lấy toàn bộ ảnh test
    all_images = []
    for class_name in os.listdir(test_path):
        class_dir = os.path.join(test_path, class_name)
        if not os.path.isdir(class_dir):
            continue
        image_files = [
            os.path.join(class_dir, f)
            for f in os.listdir(class_dir)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        all_images.extend(image_files)

    print(f"📸 Tổng số ảnh test: {len(all_images)}")

    y_true, y_pred, processing_times = [], [], []

    for img_path in tqdm(all_images, desc="Testing with SIFT+FAISS"):
        folder_name = os.path.basename(os.path.dirname(img_path))
        if folder_name not in class_to_idx:
            continue
        start_time = time.time()

        feature = extract_sift_mean_vector(img_path, sift)
        if feature is None:
            continue
        feature = feature / (np.linalg.norm(feature) + 1e-10)
        feature = feature.reshape(1, -1).astype(np.float32)

        D, I = index.search(feature, 1)
        euclidean_dist_squared = D[0][0]
        similarity_score = 1 - euclidean_dist_squared / 2

        if similarity_score < similarity_threshold:
            pred_label = len(classes)  # "Khác"
        else:
            pred_label = int(index_labels[I[0][0]]) + 1

        y_true.append(class_to_idx[folder_name])
        y_pred.append(pred_label)
        processing_times.append(time.time() - start_time)

    # === Đánh giá ===
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    accuracy = np.mean(y_true == y_pred)

    print(f"\n✅ Accuracy: {accuracy*100:.2f}%")
    print(f"✅ Đúng: {np.sum(y_true == y_pred)} / ❌ Sai: {np.sum(y_true != y_pred)}")

    avg_processing_time = np.mean(processing_times)
    print(f"⏱️ Avg time: {avg_processing_time:.4f} giây/ảnh")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    class_labels = [classes[i] for i in sorted(classes.keys())]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)
    fig, ax = plt.subplots(figsize=(16, 16))
    disp.plot(xticks_rotation=90, cmap='Blues', ax=ax, colorbar=False)
    plt.title("Confusion Matrix (FAISS)")
    plt.tight_layout()
    plt.savefig(confusion_output_path, dpi=300)
    plt.close()
    print(f"🖼️ Confusion matrix saved to {confusion_output_path}")

    # Classification report
    report = classification_report(
        y_true, y_pred, target_names=class_labels,
        digits=4, output_dict=True, zero_division=0
    )
    report_df = pd.DataFrame(report).transpose()
    report_df.to_csv(csv_output_path, index=True)
    print(f"📄 Report saved to {csv_output_path}")

    # Thống kê macro & weighted
    macro_avg = report["macro avg"]
    weighted_avg = report["weighted avg"]

    print("\n📊 Evaluation Metrics:")
    print(f"   🎯 Accuracy: {accuracy:.4f}")
    print(f"   📊 Macro Precision: {macro_avg['precision']:.4f}")
    print(f"   📊 Macro Recall: {macro_avg['recall']:.4f}")
    print(f"   📊 Macro F1-score: {macro_avg['f1-score']:.4f}")
    print(f"   📊 Weighted Precision: {weighted_avg['precision']:.4f}")
    print(f"   📊 Weighted Recall: {weighted_avg['recall']:.4f}")
    print(f"   📊 Weighted F1-score: {weighted_avg['f1-score']:.4f}")
    print(f"   🚀 Avg Processing Time: {avg_processing_time:.4f} sec/image")

    return accuracy, avg_processing_time


# ================== MAIN ==================
if __name__ == "__main__":
    # Train + FAISS
    index, index_labels = build_faiss_index(train_path, classes_train)

    # Test
    evaluate_faiss(test_path, classes_test, index, index_labels)


🧩 Trích xuất SIFT từ tập Train...


Processing Tren_Ban_Nhau_Duoi_Ban_Muu: 100%|██████████| 1020/1020 [00:13<00:00, 77.92it/s]


✅ Đã thêm 65958 vector vào FAISS.
📸 Tổng số ảnh test: 23549


Testing with SIFT+FAISS: 100%|██████████| 23549/23549 [05:56<00:00, 66.03it/s]



✅ Accuracy: 42.34%
✅ Đúng: 9944 / ❌ Sai: 13542
⏱️ Avg time: 0.0150 giây/ảnh
🖼️ Confusion matrix saved to confusion_matrix_sift_faiss.jpg
📄 Report saved to classification_report_sift_faiss.csv

📊 Evaluation Metrics:
   🎯 Accuracy: 0.4234
   📊 Macro Precision: 0.4005
   📊 Macro Recall: 0.4563
   📊 Macro F1-score: 0.4161
   📊 Weighted Precision: 0.4463
   📊 Weighted Recall: 0.4234
   📊 Weighted F1-score: 0.3887
   🚀 Avg Processing Time: 0.0150 sec/image
